### Goals

Create a model that will be able to predict who could survive and who could die in the Titanic disaster, based on selected passenger information.

### Loading libraries

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from IPython.display import display_html

from sklearn.model_selection import train_test_split, validation_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import scikitplot as skplt

from hyperopt import hp, STATUS_OK, Trials, fmin, tpe, partial

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

### Import datasets

In [ ]:
train_data = pd.read_csv('data/titanic_train.csv')
test_data = pd.read_csv('data/titanic_test.csv')

In [ ]:
train_data['IsCabin'] = train_data.Cabin.fillna(0)
train_data.loc[train_data.IsCabin != 0, 'IsCabin'] = 1
train_data.IsCabin = train_data.IsCabin.astype(int)

In [ ]:
test_data['IsCabin'] = test_data.Cabin.fillna(0)
test_data.loc[test_data.IsCabin != 0, 'IsCabin'] = 1
test_data.IsCabin = test_data.IsCabin.astype(int)

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
data = train_data.drop(columns='Survived', axis=1).append(test_data).reset_index()

In [ ]:
data.set_index('index', inplace=True)

In [ ]:
data.shape

In [ ]:
data.head()

### Basic visualization of existing data

In [ ]:
fig = plt.figure(figsize=(12, 15))
fig.suptitle("0 - No, 1 - Yes", fontsize=25, y=1.05)
plt.subplot2grid((4, 3), (0, 0))
sns.boxplot(x='Survived', y='Age', data=train_data, orient='v')
plt.title('Age')
plt.subplot2grid((4, 3), (0, 1))
sns.boxplot(x='Survived', y='Fare', data=train_data, orient='v')
plt.title('Fare')
plt.subplot2grid((4, 3), (0, 2))
sns.countplot(x='Sex', hue='Survived', data=train_data)
plt.subplot2grid((4, 3), (1, 0))
sns.countplot(x='Survived', data=train_data)
plt.title('Survived')
plt.subplot2grid((4, 3), (1, 1))
sns.countplot(x='Pclass', data=train_data)
plt.title('Pclass')
plt.subplot2grid((4, 3), (1, 2))
sns.countplot(x='Embarked', data=train_data)
plt.title('Embarked')
plt.subplot2grid((4, 3), (2, 0))
sns.countplot(x='Pclass', hue='Survived', data=train_data)
plt.title('Pclass by Survived/Died')
plt.subplot2grid((4, 3), (2, 1))
sns.countplot(x='SibSp', hue='Survived', data=train_data)
plt.title('SibSP by Survived/Died')
plt.subplot2grid((4, 3), (2, 2))
sns.countplot(x='Parch', hue='Survived', data=train_data)
plt.title('Parch by Survived/Died')
plt.subplot2grid((4, 3), (3, 0))
sns.countplot(x='IsCabin', hue='Survived', data=train_data)
plt.title('Cabin or not by Survived')

plt.tight_layout()

## Preprocessing data - version 2

In [ ]:
data_v2 = data.copy()

In [ ]:
data_v2.reset_index(inplace=True, drop=True)

In [ ]:
data_v2.head()

In [ ]:
data_v2.describe()

In [ ]:
data_v2.info()

### Handling missing values

In [ ]:
data_v2.isna().sum()

#### Mean Fare by Embarked for missing values

In [ ]:
data_v2.loc[:][data_v2.Fare.isna()]

In [ ]:
data_v2.Embarked.value_counts(dropna=False)

In [ ]:
data_v2.Fare = data_v2.Fare.fillna(data_v2.Fare[(data_v2.Embarked == 'S') & (data_v2.Pclass == 3)].mean())

In [ ]:
data_v2.Age = data_v2.Age.fillna(data_v2.Age.mean())

In [ ]:
sns.distplot(data_v2.Age, kde=False);

#### Most common Embarked value for missing data

In [ ]:
data_v2.Embarked.value_counts().idxmax()

In [ ]:
data_v2.Embarked = data_v2.Embarked.fillna(data_v2.Embarked.value_counts().idxmax())

#### Mapping values for Embarked:
S -> 1  
C -> 2  
Q -> 3  

In [ ]:
data_v2.Embarked = data_v2.Embarked.map({'S': 1, 'C': 2, 'Q': 3})

In [ ]:
data_v2.Sex = data_v2.Sex.replace('female', 0)
data_v2.Sex = data_v2.Sex.replace('male', 1)
data_v2.Sex.astype(int)

#### Creating Fare Per Passenger Attribute

In [ ]:
sns.distplot(data_v2.Fare, kde=False, norm_hist=True);

In [ ]:
ticket_counts = data_v2.Ticket.value_counts()
tickets_index = ticket_counts.index

In [ ]:
tempor_tickets = pd.Series(ticket_counts, index=tickets_index)
tempor_fares = pd.Series(data_v2.groupby('Ticket')['Fare'].mean(), index=tickets_index)

In [ ]:
fare_pp = pd.DataFrame({'Count': ticket_counts,
                           'Fare': tempor_fares}, index=tickets_index)

In [ ]:
fare_pp['FarePerPassenger'] = fare_pp.apply(lambda x: x.Fare/x.Count, axis=1)
fare_pp.index = fare_pp.index.rename('Ticket')

In [ ]:
fare_pp.head()

In [ ]:
data_v2 = data_v2.join(fare_pp.loc[:,['FarePerPassenger']], on='Ticket')
data_v2.reset_index(inplace=True)

In [ ]:
data_v2.head()

## Pre-processing data - version 1

#### Preparing datasets to making predictions

In [ ]:
survived_train = train_data.Survived
data_v1 = train_data.drop('Survived', axis=1).append(test_data, sort=False)

In [ ]:
data_v1.head()

In [ ]:
data_v1.describe()

#### Mark cabins by its type and missing values by "noCabin"

In [ ]:
data_v1.Cabin = data.Cabin.str[0]
data_v1.Cabin = data.Cabin.fillna('noCabin')

#### Changing values of sex variable to 0 (female) and 1 (male)

In [ ]:
data_v1.Sex = data_v1.Sex.replace('female', 0)
data_v1.Sex = data_v1.Sex.replace('male', 1)
data_v1.Sex= data_v1.Sex.astype(int)

#### How big a family a passenger has and what is the distribution of this variable?

In [ ]:
data_v1['FamilySize'] = data_v1.SibSp + data_v1.Parch + 1
data_v1['IsSolo'] = 0
data_v1.loc[data_v1.FamilySize == 1, 'IsSolo'] = 1
data_v1['SmallGroup'] = 0
data_v1.loc[data_v1.FamilySize.isin([2, 3, 4]), 'SmallGroup'] = 1
sns.countplot(data_v1['FamilySize']);

#### Extracting titles of passengers and presenting distribution of age by title

In [ ]:
data_v1['Title'] = data_v1['Name'].str.replace('(.*, )|(\. .*)', '')
data_v1.Title = data_v1.Title.replace(['Mme', 'Ms', 'Lady', 'Mlle', 'the Countess', 'Dona'], 'lady')
data_v1.Title = data_v1.Title.replace(['Don', 'Major', 'Sir', 'Col', 'Capt', 'Jonkheer'], 'other')

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot2grid((1, 3), (0, 0))
sns.boxplot(x='Title', y='Age', data=data_v1);

#### If the age is unknown, the passenger receives the average age from the title category to which he belongs

In [ ]:
for title in data_v1.Title.unique().tolist():
    data_v1.loc[data_v1.Title == title, ['Age']] = data_v1.loc[data_v1.Title == title, ['Age']].fillna(data_v1.loc[data_v1.Title == title, ['Age']].mean())

#### Calculating fare per passenger and presenting its distribution in the dataset (median for missing values)

In [ ]:
tickets = set(data_v1.Ticket)
ticket_numbers = []
fares = []

for ticket in tickets:
    ticket_numbers.append(len(data_v1[data_v1.Ticket == ticket]))
    fares.append(data_v1['Fare'][data_v1.Ticket == ticket].mean())

tickets_sum = pd.DataFrame()
tickets_sum['Ticket'] = list(tickets)
tickets_sum['Count'] = ticket_numbers
tickets_sum['Fare'] = fares
tickets_sum['FarePP'] = tickets_sum['Fare']/tickets_sum['Count']
tickets_sum = tickets_sum.drop(['Count', 'Fare'], axis=1)

data_v1 = pd.merge(data_v1, tickets_sum, 'left', 'Ticket')

In [ ]:
data_v1.FarePP = data_v1.FarePP.fillna(data_v1.FarePP.median())

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(a=data_v1.FarePP, kde=False);

In [ ]:
data_v1.head()

## Feature selection

In [ ]:
orig_feats = ['PassengerId', 'SibSp', 'Parch', 'FarePP', 'FamilySize']
dummy_feats = ['Sex', 'Pclass', 'IsSolo', 'SmallGroup', 'Title']

data_v2_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'IsCabin']

In [ ]:
orig_df = data_v1.loc[:, orig_feats]
dummy_df = pd.get_dummies(data_v1.loc[:, dummy_feats])

### Setting features and data

In [ ]:
# df = data_v2.loc[:, data_v2_features]
df = pd.concat([orig_df, dummy_df], axis=1)

In [ ]:
df.loc[890:,]

#### Splitting dataset to train, validation and submission subsets

In [ ]:
X = df.loc[:890,]
y = survived_train
test_x = df.loc[891:, ]

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(X, y, random_state=0)

## Modeling

#### Auxiliary functions

In [ ]:
def get_measures(y, pred_y):
    score_test = roc_auc_score(y, pred_y)
    gini_index = 2*score_test-1
    
    df = pd.DataFrame({'AUC': [round(score_test, 4)], 'Gini': [round(gini_index, 4)]})
    return df

def calculating_metrics(model, test_x, val_x, test_y, val_y):
    test = get_measures(test_y, model.predict_proba(test_x)[:, 1])
    val = get_measures(val_y, model.predict_proba(val_x)[:, 1])
    
    return pd.concat([test, val]).set_index([pd.Index(['TRAIN', 'VAL'])])

def get_performance_measure(y, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)): 
        if y[i]==y_pred[i] and y_pred[i]==1:
           TP += 1
        if y_pred[i]==1 and y[i]!=y_pred[i]:
           FP += 1
        if y[i]==y_pred[i] and y_pred[i]==0:
           TN += 1
        if y_pred[i]==0 and y[i]!=y_pred[i]:
           FN += 1
    return(TP, FP, TN, FN)

def print_indicators(real_y, predict_y):
    TP, FP, TN, FN = get_performance_measure(real_y, predict_y)
    

    pos = TP + FN
    neg = TN + FP

    tpr = TP / (TP + FN)
    tnr = TN / (TN + FP)
    fpr = FP / (FP + TN)
    fnr = FN / (FN + TP)

    acc = (TP + TN) / (TP + TN + FP + FN)

    msg = (f'Number of positive observations:\t\t{pos}\n'
           f'Number of negative observations:\t\t{neg}\n'
           f'Total number of observations:\t\t\t{pos+neg}\n\n'
           f'TPR (True Positive Rate), SE (sensitivity):\t{tpr:.4f}\n'
           f'TNR (True Negative Rate), SPC (specificity):\t{tnr:.4f}\n'
           f'FPR (False Positive Rate):\t\t\t{fpr:.4f}\n'
           f'FNR (False Negative Rate):\t\t\t{fnr:.4f}\n'
           f'ACC (Total Accuracy):\t\t\t\t{acc:.4f}'
          )
    return msg

### Decision Tree

In [ ]:
model_DTC = DecisionTreeClassifier(random_state=1)

#### Finding the best parameters for Decision Tree model using Grid Search

In [ ]:
grid_random_param = {
    'max_depth': np.linspace(1, 15, 15),
    'min_samples_split': np.linspace(0.01, 1, 10),
    'min_samples_leaf': np.linspace(0.01, 0.5, 5),
    'max_features': list(range(1, train_x.shape[1])),  
}

In [ ]:
grid = GridSearchCV(model_DTC, grid_random_param, scoring='roc_auc')
grid.fit(train_x, train_y)
best_parameters_DTC_grid = grid.best_params_
print(best_parameters_DTC_grid)

#### Train model with the best-chosen parameters, predict values and get basic metrics

In [ ]:
model_GS = DecisionTreeClassifier(**best_parameters_DTC_grid, random_state=1)
model_GS.fit(train_x, train_y)
cm_DTC_GS = calculating_metrics(model_GS, train_x, val_x, train_y, val_y)
cm_DTC_GS

#### Confusion matrix

In [ ]:
c_matrix_dtc_gs = confusion_matrix(val_y, model_GS.predict(val_x))
print(c_matrix_dtc_gs)

#### Measures based on a confusion matrix

In [ ]:
real_y = val_y.to_numpy(dtype=int)
predict_y = model_GS.predict(val_x)

In [ ]:
print(f'\nTN:\t{c_matrix_dtc_gs[0][0]}\nFN:\t{c_matrix_dtc_gs[1][0]}\nTP:\t{c_matrix_dtc_gs[1][1]}\nFP:\t{c_matrix_dtc_gs[0][1]}\n')
print(print_indicators(real_y, predict_y))

#### AUC chart

In [ ]:
pred_prob = model_GS.predict_proba(val_x)
fpr_dtc_gs, tpr_dtc_gs, th = roc_curve(val_y, pred_prob[:, 1])
auc_dts_gs = roc_auc_score(val_y, pred_prob[:, 1])

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(fpr_dtc_gs, tpr_dtc_gs, 'o-')
plt.ylabel('TPR')
plt.xlabel('FPR')
print(f'AUC: {auc_dts_gs:.4f}')

#### LIFT chart

In [ ]:
skplt.metrics.plot_lift_curve(val_y, pred_prob);

#### Finding the best parameters for Decision Tree model using Random Search

In [ ]:
random_search = RandomizedSearchCV(model_DTC, grid_random_param, n_iter=200, scoring='roc_auc')
random_search.fit(train_x, train_y)
best_parameters_DTC_random = random_search.best_params_
print(best_parameters_DTC_random)

#### Train model with the best-chosen parameters, predict values and get basic metrics

In [ ]:
model_RS = DecisionTreeClassifier(**best_parameters_DTC_random, random_state=1)
model_RS.fit(train_x, train_y)
cm_DTC_RS = calculating_metrics(model_RS, train_x, val_x, train_y, val_y)
cm_DTC_RS

#### Finding the best parameters for Decision Tree model using Baysian optimalization

In [ ]:
n_startup_jobs = 80
max_evals = 500
BS_results = []

In [ ]:
b_opt_space = {
    'max_depth': hp.quniform('max_depth', 1, 15, 1),
    'min_samples_split': hp.uniform('min_samples_split', 0.01, 1),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.01, 0.5),
    'max_features': hp.quniform('max_features', 1, train_x.shape[1], 1)
}

In [ ]:
def objective(space):
    b_params = {
        'max_depth': int(space['max_depth']),
        'min_samples_split': space['min_samples_split'],
        'min_samples_leaf': space['min_samples_leaf'],
        'max_features': int(space['max_features'])
    }
    model = DecisionTreeClassifier(**b_params, random_state=0)
    model.fit(train_x, train_y)
    pred_y = model.predict(val_x)
    
    score = -roc_auc_score(val_y, pred_y)
    return {'loss': score, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best_parameters_DTC_bayesian = fmin(fn=objective,
                   space=b_opt_space, 
                   algo=partial(tpe.suggest, n_startup_jobs=n_startup_jobs),
                   max_evals=max_evals,
                   trials=trials
                  )
print(f'best params: {best_parameters_DTC_bayesian}')

In [ ]:
best_parameters_DTC_bayesian['max_features'] = int(best_parameters_DTC_bayesian['max_features'])

#### Train model with the best-chosen parameters, predict values and get basic metrics

In [ ]:
model_BO = DecisionTreeClassifier(**best_parameters_DTC_bayesian, random_state=1)
model_BO.fit(train_x, train_y)
cm_DTC_BO = calculating_metrics(model_BO, train_x, val_x, train_y, val_y)
cm_DTC_BO

#### Comparison of prediction results with using Decision Tree model

In [ ]:
results_DTC = pd.concat([cm_DTC_GS, cm_DTC_RS, cm_DTC_BO],
                        axis=1,
                        keys={'GidSearchCV': cm_DTC_GS,
                              'RandomSearchCV': cm_DTC_RS,
                              'Bayesian Optimization': cm_DTC_BO}
                       )
results_DTC

### Random Forest

In [ ]:
model_RFC = RandomForestClassifier(random_state=1)

#### Finding the best parameters for Random Forest model using Grid Search

In [ ]:
grid_random_param_RFC = {
    'n_estimators': [200, 250, 300],
    'max_depth': [20, None],
    'min_samples_split': [5, 10, 12],
    'min_samples_leaf': [2, 3, 4, 5]
}

In [ ]:
grid_RFC_GS = GridSearchCV(model_RFC, grid_random_param_RFC, scoring='roc_auc')
grid_RFC_GS.fit(train_x, train_y)
best_parameters_RFC_grid = grid_RFC_GS.best_params_
print(best_parameters_RFC_grid)

#### Train model with the best-chosen parameters, predict values and get basic metrics

In [ ]:
model_RFC_GS = RandomForestClassifier(**best_parameters_RFC_grid, random_state=1)
model_RFC_GS.fit(train_x, train_y)
cm_RFC_GS = calculating_metrics(model_RFC_GS, train_x, val_x, train_y, val_y)
cm_RFC_GS

#### Finding the best parameters for Random Forest model using Random Search

In [ ]:
random_RFC_RS = RandomizedSearchCV(model_RFC, grid_random_param_RFC, scoring='roc_auc')
random_RFC_RS.fit(train_x, train_y)
best_parameters_RFC_random = random_RFC_RS.best_params_
print(best_parameters_RFC_random)

#### Train model with the best-chosen parameters, predict values and get basic metrics

In [ ]:
model_RFC_RS = RandomForestClassifier(**best_parameters_RFC_random, random_state=1)
model_RFC_RS.fit(train_x, train_y)
cm_RFC_RS = calculating_metrics(model_RFC_RS, train_x, val_x, train_y, val_y)
cm_RFC_RS

#### Finding the best parameters for Random Forest model using Baysian optimalization

In [ ]:
b_opt_space = {
    'n_estimators': hp.quniform('n_estimators', 200, 300, 50),
    'max_depth': hp.quniform('max_depth', 10, 30, 2),
    'min_samples_split': hp.quniform('min_samples_split', 4, 116, 2),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 8, 2)
}

In [ ]:
def objective(space):
    b_opt_space = {
        'n_estimators': int(space['n_estimators']),
        'max_depth': int(space['max_depth']),
        'min_samples_split': int(space['min_samples_split']),
        'min_samples_leaf': int(space['min_samples_leaf']),
    }
    model = RandomForestClassifier(**b_opt_space, random_state=0)
    model.fit(train_x, train_y)
    pred_y = model.predict(val_x)
    score = -roc_auc_score(val_y, pred_y)
    return {'loss': score, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best_parameters_RFC_bayesian = fmin(fn=objective,
                                    space=b_opt_space,
                                    algo=partial(tpe.suggest, n_startup_jobs=n_startup_jobs),
                                    max_evals=max_evals,
                                    trials=trials
                                   )
print(f'best params: {best_parameters_RFC_bayesian}')

In [ ]:
best_parameters_RFC_bayesian['n_estimators'] = int(best_parameters_RFC_bayesian['n_estimators'])
best_parameters_RFC_bayesian['max_depth'] = int(best_parameters_RFC_bayesian['max_depth'])
best_parameters_RFC_bayesian['min_samples_split'] = int(best_parameters_RFC_bayesian['min_samples_split'])
best_parameters_RFC_bayesian['min_samples_leaf'] = int(best_parameters_RFC_bayesian['min_samples_leaf'])

#### Train model with the best-chosen parameters, predict values and get basic metrics

In [ ]:
model_RFC_BO = RandomForestClassifier(**best_parameters_RFC_bayesian, random_state=0)
model_RFC_BO.fit(train_x, train_y)
cm_RFC_BO = calculating_metrics(model_RFC_BO, train_x, val_x, train_y, val_y)
cm_RFC_BO

#### Comparison of prediction results with using Random Forest model

In [ ]:
results_RFC = pd.concat([cm_RFC_GS, cm_RFC_RS, cm_RFC_BO],
                        axis=1,
                        keys={'GidSearchCV': cm_RFC_GS,
                              'RandomSearchCV': cm_RFC_RS,
                              'Bayesian Optimization': cm_RFC_BO}
                       )
results_RFC

### Results summary

In [ ]:
results_DTC.index = pd.MultiIndex.from_product([['Decision Tree'], ['Train', 'Val']])
results_RFC.index = pd.MultiIndex.from_product([['Random Forest'], ['Train', 'Val']])

In [ ]:
results_summ = results_DTC.append(results_RFC)
results_summ

### KNN (K nearest neighbour)

In [ ]:
scale = MinMaxScaler()

In [ ]:
X_scaled = pd.DataFrame(scale.fit_transform(X), columns=X.columns)

In [ ]:
test_x_scaled = pd.DataFrame(scale.fit_transform(test_x), columns=test_x.columns)
train_x_s, val_x_s, train_y_s, val_y_s = train_test_split(X_scaled, y, random_state=1)

In [ ]:
for i in range(1, 20):
    knn_model = KNeighborsClassifier(n_neighbors=i).fit(train_x_s, train_y_s)
    print(knn_model.score(val_x_s, val_y_s))

In [ ]:
calculating_metrics(KNeighborsClassifier(n_neighbors=11).fit(val_x_s, val_y_s), train_x_s, val_x_s, train_y_s, val_y_s)

In [ ]:
train_score, test_score = validation_curve(KNeighborsClassifier(),
                                           X,
                                           y,
                                           param_name='n_neighbors',
                                           param_range=range(1, 21),
                                           cv=5
                                          )

In [ ]:
train_score_mean = np.mean(train_score, axis=1)
test_score_mean = np.mean(test_score, axis=1)

In [ ]:
plt.figure(figsize=(6, 6))
sns.set_style('ticks')
sns.lineplot(x=range(1,21), y=train_score_mean, markers=True, marker='o', )
sns.lineplot(x=range(1,21), y=test_score_mean, markers=True, marker='o')
plt.legend(['train score', 'test score'])
plt.xlabel('n_neighbors')
plt.ylabel('Accuracy');

## Submission

In [ ]:
subm = pd.read_csv('data/gender_submission.csv')

In [ ]:
pred_values = model_RFC_BO.predict(test_x)

In [ ]:
subm_values = pd.Series(pred_values, name='Survived', index=range(len(pred_values)))

In [ ]:
subm.Survived = subm_values

In [ ]:
subm.to_csv('data/gender_submission.csv', index=False)